# Machine Learning and Classification 
## Rapport du projet

*Par BENAMIRA Adrien, CARRIE Hanaé et DEVILLERS Benjamin*

## 1. Introduction

blabla à propos de l'obj...

## 2. Chargement des données et pré-traitements

### 2.1. Chargement des données

Les données sont sauvegardée au format HDF5. Ce format est particulièrement intéressant car il permet de stocker des lourdes données compréssées en un dataset. La bibliothèque Python `h5py` permet de pouvoir les lire ces données facilement de la même manière que nous utiliserions un tableau `Numpy`.

La class python `DreemDataset` dans le package `tools.data` permet de récupérer et manipuler les datasets.
Comme nous voulons effectuer des prétraitements sur nos données et les stocker pour pouvoir rapidement appliquer les algorithmes d'apprentissage sur nos données traitées, nous avons conçu cette classe de sorte que nous puissions appliquer puis sauvegarder.

Nous avons choisis de les sauvegarder ensuite dans des tableaux `npy` pour des questions de performances d'execution.

La classe python `DreemDatasets` permet quant à elle de générer un "dev set" et "train set" à partir des données. Celle-ci retourne un couple d'instance de `DreemDataset`. Puis que les différentes classes ne sont pas équilibrées, nous pouvons également ne récupérer qu'un sous-ensemble du dataset équilibré.

In [1]:
from tools.data import DreemDatasets

Les paramètres sont :

- chemin vers les données
- chemin vers les cibles
- `keep_datasets` Liste des datasets à garder, parmi les suivants :
    * `eeg_1` - EEG in frontal position sampled at 50 Hz -> 1500 values
    * `eeg_2` - EEG in frontal position sampled at 50 Hz -> 1500 values
    * `eeg_3` - EEG in frontal position sampled at 50 Hz -> 1500 values
    * `eeg_4` - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
    * `eeg_5` - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
    * `eeg_6` - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
    * `eeg_7` - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
    * `accelerometer_x` - Accelerometer along x axis sampled at 10 Hz -> 300 values
    * `accelerometer_y` - Accelerometer along y axis sampled at 10 Hz -> 300 values
    * `accelerometer_z` - Accelerometer along z axis sampled at 10 Hz -> 300 values
    * `pulse_oximeter_infrared` - Pulse oximeter infrared channel sampled at 10 Hz -> 300 values
- `split_train_val` Pourcentage pour partager le train set et validation set
- `seed` Une seed pour la reproductibilité
- `balance_data` si vrai, équilibre le dataset pour avoir le nombre de donnée par classe
- `size` Une taille maximale pour le dataset (si non renseignée, tout le dataset)
- `transforms` des transformations à appliquer aux données (voir la partie transformation)
- `transforms_val` si renseignée, `transforms` sera pour le train et `transforms_val` pour la validation. Sinon, même transformation que `transforms`.

In [2]:
train_set, val_set = DreemDatasets('dataset/train.h5', 
                                   'dataset/train_y.csv', 
                                   split_train_val=0.8, 
                                   seed=0, 
                                   keep_datasets=['eeg_1']).get()

train_set.load_data()  # Load les données en mémoire

val_set.load_data()

# Ne pas oublier de fermer les datasets
# Ne ferme que les fichiers h5. Si .load_data() a été appelé, on a toujours accès aux données !
train_set.close()
val_set.close()

Loading data in memory...
5412 in 1 datasets to load
Loading dataset eeg_1 ...
Done.
Loading data in memory...
1353 in 1 datasets to load
Loading dataset eeg_1 ...
Done.


#### 2.1.1. Récupération des données

In [7]:
data_50hz, data_10hz, target = train_set[0]  # Une valeur

# Dimension nb_datasets x tailles_features
print(data_50hz.shape)

data_50hz, data_10hz, targets = train_set[:10]  # 10 valeurs
# Dimension nb_datasets x nb_elements (=10) x tailles_features

print(data_50hz.shape)

(1, 1500)
(1, 10, 1500)


#### 2.1.2. Enregistrer un dataset

On peut utiliser la méthode `save_data` pour enregistrer dans un `.npy`. Pour des questions de mémoire, il y aura un fichier par dataset (ex, si on choisit d'ouvrir `["eeg_1", "eeg_2"]` alors il y aura deux fichiers).

Attention, le nom des fichiers ne peut pas être précisé, seulement un dossier parent.

Par exemple, avec le `save_data("dataset/test")`, on aura les fichiers :
- `dataset/test/eeg_1.npy`
- `dataset/test/eeg_2.npy`

On peut préciser le chemin vers un nouveau dossier, celui-ci sera créé.

La sauvegarde enregistre les données **transformée**. Les données brutes ne sont pas enregistrées.

In [8]:
train_set, val_set = DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', 
                                   split_train_val=0.8, seed=0, keep_datasets=['eeg_1']).get()

train_set.save_data("dataset/sauvegarde/train")  # Attention, pas de / à la fin !
val_set.save_data("dataset/sauvegarde/val")

train_set.close()
val_set.close()

Saving into dataset/sauvegarde/train ...
Loading dataset eeg_1 ...
Saved.
Saving into dataset/sauvegarde/val ...
Loading dataset eeg_1 ...
Saved.


Si l'on a déjà enregistré le dataset, on peut utiliser `load_data` en précisant le dossier dans lequel sont tous les fichiers

In [10]:
train_set, val_set = DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', 
                                   split_train_val=0.8, seed=0, keep_datasets=['eeg_1']).get()

# Cela remplace les données données des fichiers h5.
train_set.load_data("dataset/sauvegarde/train")  # Attention, pas de / à la fin !
val_set.load_data("dataset/sauvegarde/val")

train_set.close()
val_set.close()

data_50hz, data_10hz, target = train_set[0]

print(data_50hz.shape)

Loading data in memory...
5412 in 1 datasets to load
Loading dataset eeg_1 ...
Done.
Loading data in memory...
1353 in 1 datasets to load
Loading dataset eeg_1 ...
Done.
(1, 1500)


#### 2.1.3. Test set

On peut faire la même chose avec le test set qu'avec les autres données.

Attention à bien appeler `init()`.

In [12]:
from tools.data import DreemDataset

test_set = DreemDataset('dataset/test.h5', keep_datasets=['eeg_1']).init()

# Rappel : charge en mémoire. On peut également charget un fichier en précisant un chemin
test_set.load_data()  

test_set.close()

Loading data in memory...
37439 in 1 datasets to load
Loading dataset eeg_1 ...
Done.


### 2.2. Pré-traitements

#### 2.2.1. Introduction : Transformations

Une transformation est ce qui nous permet de faire nos pré-traitements sur nos données.
Pour effectuer des transformations sur nos données, nous devons definir un dictionnaire :

In [4]:
def transformation_eeg_1(batch_signals, batch_targets):
    return batch_signals[:, 0]

transformations = {
    "eeg_1": transformation_eeg_1
}

Prenant en clé le nom du dataset sur lequel effectuer la transformation et en valeur une fonction prenant 2 paramètres :

* un batch du signal de taille (batch, taille_signal)
* les "targets" qui correspondent aux classes de sommeils que nous devons prédire pour le batch en question.

Ces deux paramètres peuvent être utiles pour nos différentes fonctions de pré-processing.

La transformation doit retourner le batch modifié.

Dans l'exemple ci-dessus, nous appliquerons une transformation au dataset `eeg_1` qui tranformera les signaux en seulement la première valeur du signal.

#### 2.2.2. Extraction en bandes

D'après [[1: Malhotra, R. K., & Avidan, A. Y. (2014). Sleep stages and scoring technique. Atlas of Sleep Medicine, 77-99.](http://www.fmed.edu.uy/sites/www.labsueno.fmed.edu.uy/files/9.Diagnostico-polisomnogr%C3%A1fico.pdf)] et [[2: Aboalayon, K., Faezipour, M., Almuhammadi, W., & Moslehpour, S. (2016). Sleep stage classification using EEG signal analysis: a comprehensive survey and new investigation. Entropy, 18(9), 272.](https://www.mdpi.com/1099-4300/18/9/272)], les électroencéphalogrames (eegs) peuvent être séparés en bande de fréquences contenant différentes informations :

* Bande $\delta$ : de 0 à 4 Hz,
* bande $\theta$ : de 4 à 8 Hz,
* bande $\alpha$ : de 8 à 13 Hz,
* bande $\beta$ : de 13 à 38 Hz,
* bande $\gamma$ : de 38 à 42 Hz.

Cependant nos eegs sont échantillonés à 50 Hz. Le thèorème de Shannon (Sampling theorem) indique que nous ne pourrons récupérer que des informations sur une bande de fréquence entre 0 et 25 Hz.

Nous ne considerons donc pour cela pas la bande $\gamma$ et gardon la bande $\beta$ que de 13 à 22 Hz.

In [13]:
from preprocessing.signals import ExtractBands

In [14]:
extract_bands = ExtractBands(bands='*')  # toutes les bandes (de delta à beta)

`extract_bands` est une transformation que nous pouvons appliquer aux eegs et qui sort des signaux extraits en bande :

In [16]:
train_set, val_set = DreemDatasets('dataset/train.h5', 
                                   'dataset/train_y.csv', 
                                   split_train_val=0.8, 
                                   seed=0, 
                                   keep_datasets=['eeg_1'],
                                   transforms={"eeg_1": extract_bands}).get()

train_set.load_data()  # Load les données en mémoire

val_set.load_data()

# Ne pas oublier de fermer les datasets
# Ne ferme que les fichiers h5. Si .load_data() a été appelé, on a toujours accès aux données !
train_set.close()
val_set.close()

data_50hz, data_10hz, target = train_set[0]

print(data_50hz.shape)

Loading data in memory...
5412 in 1 datasets to load
Loading dataset eeg_1 ...
Apply transformations...
Applied.
Done.
Loading data in memory...
1353 in 1 datasets to load
Loading dataset eeg_1 ...
Apply transformations...
Applied.
Done.
(1, 4, 1500)


Noter la dimension "4" ajoutée qui correspond aux 4 bandes de fréquence.

#### 2.2.3. Features plus avancées

Nous avons ensuite ajouté différents pré-traitements de base sur nos features tel que :

* `min` valeur min du signal,
* `max` valeur max du signal,
* `frequency` fréquence du signal,
* `energy` énergie du signal.

Puis nous avons essayé les features proposés par [[2](https://www.mdpi.com/1099-4300/18/9/272)] :
* `mmd` distance minimum-maximum
* `esis` correspond à l'énergie et "vitesse" du signal

De plus, comme expliqué par [[1](http://www.fmed.edu.uy/sites/www.labsueno.fmed.edu.uy/files/9.Diagnostico-polisomnogr%C3%A1fico.pdf)], section "_Stages of Sleep_", nous pouvons distinguer différentes étapes du sommeil par différentes prédominances "d'utilisation" d'une bande :
* `band-usage` est la feature qui indique, sur une fenêtre glissante, la bande qui est a la plus grande valeur sur la fenêtre parmi toutes les bandes. Cela donne une indication de la bande prédominante sur la fenêtre.

In [18]:
# Transformation pour ces featyres
from preprocessing.features import ExtractFeatures

# Extract features peut directement extraire les bandes
extract_features = ExtractFeatures(bands='*', features=['mmd', 'esis'])

In [27]:
train_set, val_set = DreemDatasets('dataset/train.h5', 
                                   'dataset/train_y.csv', 
                                   split_train_val=0.8, 
                                   seed=0, 
                                   keep_datasets=['eeg_1'],
                                   transforms={"eeg_1": extract_features}).get()

train_set.load_data()  # Load les données en mémoire

val_set.load_data()

# Ne pas oublier de fermer les datasets
# Ne ferme que les fichiers h5. Si .load_data() a été appelé, on a toujours accès aux données !
train_set.close()
val_set.close()

data_50hz, data_10hz, target = train_set[:]

print(data_50hz.shape)

Loading data in memory...
5412 in 1 datasets to load
Loading dataset eeg_1 ...
Apply transformations...
Applied.
Done.
Loading data in memory...
1353 in 1 datasets to load
Loading dataset eeg_1 ...
Apply transformations...
Applied.
Done.
(1, 4, 5412, 2)


On voit ici que la dimension est de 1 (=1 dataset) x 4 (=4 bandes) x nombre de signaux x 2 (=2 features).